In [93]:
# -*- coding: utf-8 -*- 
import urllib , requests , sys ,string ,time
from bs4 import BeautifulSoup
from bs4 import SoupStrainer
from time import gmtime, strftime

#dmm
#fkey作者來源key.txt
#fcomiclist結果key_dmm.text

#輸出格式：
#dmm
#!作者
#!總筆數
#==類別_數量_中文敘述
#[同人團_作者群][原作][同人場cXX]同人作品
#參考網址

#頁顯示數量
pnum = 30
mlink = 'http://book.dmm.co.jp/'

fkey = open('key.txt', 'r')
key = fkey.readline()#key=作者
fkey.close()

#key=作者
key2 = urllib.quote(key.decode('utf8').encode('shift_jis'))#當sjis輸出utf8的url
key3 = urllib.unquote(key2.decode('shift_jis').encode('utf8'))#utf8的url翻sjis
#網址用
key4 = urllib.quote(key)

#檢查BOM
if '%EF%BB%BF' in urllib.quote(key):
    print 'fuck ms'

'''
http://www.dmm.co.jp/search/=
/searchstr=%E5%A5%A5%E6%A3%AE%E3%83%9C%E3%82%A6%E3%82%A4#作者
/analyze=V1EBD1YFUAM_#可省略?
/limit=30#頁顯示數量
/n1=FgRCTw9VBA4GAFNXXF0_
/sort=date#排序
/view=text#檢視模式_有無
/page=2#頁_P1沒有
/#尾
'''
#page=1
link="http://www.dmm.co.jp/search/=/\
searchstr="+key4+"/limit=30/n1=FgRCTw9VBA4GAFNXXF0_/sort=date/"
#標題與話數_http://book.dmm.co.jp/detail/b333afjpc00714/

res = requests.get(link)
res.encoding =  res.apparent_encoding#亂碼處理
r= res.text
#r=r[r.find(u'<td id="mu">'):r.find(u'<!-- /mu --></td>')+len(u'<!-- /mu --></td>')]#+'</td>'#搜尋結果擷取
r=r[r.find(u'</div><![endif]-->')+len(u'</div><![endif]-->'):r.find(u'<div id="footer">')]#+'</td>'#搜尋結果擷取


#only_a_tags = SoupStrainer(id="mu")#縮小檢索範圍
soup = BeautifulSoup(r,"lxml")#,  parse_only=only_a_tags)

def next(page = 2):
    #page=int(page)
    #ps=page*pnum+1
    link="http://www.dmm.co.jp/search/=/\
    searchstr="+key4+"/limit=30/n1=FgRCTw9VBA4GAFNXXF0_/sort=date/"
    
    if page>1:
        link=link+'page='+str(page)+'/'
    
    res = requests.get(link)
    res.encoding =  res.apparent_encoding
    r= res.text
    r=r[r.find(u'</div><![endif]-->')+len(u'</div><![endif]-->'):r.find(u'<div id="footer">')]#+'</td>'#搜尋結果擷取
    
    #先html.parser解析與縮小範圍，再以字串給lxml
    #only_a_tags = SoupStrainer(id="mu")#縮小檢索範圍
    soup = BeautifulSoup(r,"lxml")#,  parse_only=only_a_tags)
    
    return soup

#全轉半
def Q2B(ustring):
    fs=u'０１２３４５６７８９ＱｑＷｗＥｅＲｒＴｔＹｙＵｕＩｉＯｏＰｐＡａＳｓＤｄＦｆＧｇＨｈＪｊＫｋＬｌＺｚＸｘＣｃＶｖＢｂＮｎＭｍ'
    hs=u'0123456789QqWwEeRrTtYyUuIiOoPpAaSsDdFfGgHhJjKkLlZzXxCcVvBbNnMm'
    rstr = ''
    for tm in ustring:
        if fs.find(tm)+1:
            tm = hs[fs.find(tm)]
        rstr = rstr + tm
    ustring = rstr
    ustring=ustring.lower()
    return ustring
#半轉全
def B2Q(ustring):
    fs=u'０１２３４５６７８９ＱｑＷｗＥｅＲｒＴｔＹｙＵｕＩｉＯｏＰｐＡａＳｓＤｄＦｆＧｇＨｈＪｊＫｋＬｌＺｚＸｘＣｃＶｖＢｂＮｎＭｍ'
    hs=u'0123456789QqWwEeRrTtYyUuIiOoPpAaSsDdFfGgHhJjKkLlZzXxCcVvBbNnMm'
    rstr = ''
    ustring=ustring.lower()
    for tm in ustring:
        if hs.find(tm)+1:
            tm = fs[hs.find(tm)]
        rstr = rstr + tm
    ustring = rstr
    return ustring

#資料儲存
def save(sdict , check=0):
    for temp in listdata:
        if sdict.get(temp):
            fout.write(sdict[temp].encode('utf8')  + '\n')
    #return

#作品頁面
def finddata(dlink):
    res = requests.get(dlink)#,cookies=cookies)
    res.encoding =  res.apparent_encoding#亂碼處理
    r= res.text
    r=r[r.find(u'</div><![endif]-->')+len(u'</div><![endif]-->'):r.find(u'<div id="footer">')]#+'</td>'#搜尋結果擷取
    
    #only_a_tags = SoupStrainer("table", summary="Details")#縮小檢索範圍
    sou=BeautifulSoup(r,"lxml")#,  parse_only=only_a_tags)
    #dname=sou.select('.m-boxDetailProductInfoMainList__description__list')[0].text
    
    dname=u''
    dbook=u''
    dcheck=0
    for d in sou.select('.m-boxDetailProductInfoMainList__description__list')[0].select('a'):
        if d.text==key.decode('utf8'):
            dcheck=1
            #print 'd'
        dname=dname+'_'+d.text
    #print 'find',dbook,dname
    dname=dname.strip(u'_')
    dbook=sou.select('.m-boxDetailProductInfoMainList__description__list')[1].text.strip().strip(u'\n').strip(u'\t')
    if dcheck==1 :
        if key.decode('utf8') != dname:
            dcheck=3
            #print 'dd'
    '''
    dname=sou.select('.DetailData_L')[2].text#作者
    draw=sou.select('.DetailData_L')[3].text#同人原作
    dtype=sou.select('.DetailData_R')[0].text#類型
    ddate=sou.select('.DetailData_R')[1].text#日期
    
    dname=Q2B(dname.strip().strip(u'\n').replace('\n',' ').replace('\t',''))
    draw=draw.strip().strip(u'\n').replace('\n','').replace('\t','')
    #print dname,draw,dtype,ddate
    #'''
    return dname,dbook,dcheck

#資料處理
def findbook(soup , page = 1):
    global bl
    a =0
    check = 0
    for n in soup.select('.m-boxListBookProduct__item'):
        cbook=n.select('.m-boxListBookProductTmb__ttl')[0].text
        ctype=n.select('.m-boxListGenreIco.m-boxListGenreIco--large')[0].text
        cbn=''#卷
        cname=Q2B(n.select('.m-boxListBookProductTmb__linkAuthor')[0].text.strip().strip(u'\n'))
        #網址
        clink=n.select('.m-boxListBookProductTmb')[0].select('a')[0].get('href')
        
        a=a+1
        print '\r',a,
        
        #類別
        #if (u'' == ctype):
        if u'単行本' == ctype:
            check=1#單行本
        elif u'単話' ==ctype:
            check=2#單話
        else:
            continue#目標外
        
        #作品頁_1標題2作者3系列
        if (u'...' in cbook)or(key.decode('utf8') !=cname):#or():
            #print 'findbook'
            cname,cbook,dcheck=finddata(clink)
            #print dcheck,'D'
            if dcheck>2:
                check=3#多作者
            elif dcheck==0:
                continue#非作者
        
        #作品名
        cbook=cbook[:cbook.rfind(u'（単話）')]
        cbook=cbook[:cbook.rfind(u'【フルカラー】')]
        
        #卷
        if n.select('.m-boxListBookProductTmbSub.m-boxListBookTmbSubInfo--series'):
            cbn=n.select('.m-boxListBookProductTmbSub.m-boxListBookTmbSubInfo--series')[0].text
            cbn=cbn[2:cbn.find(u'卷')]    
        
        cdata=clink[len(u'http://book.dmm.co.jp/detail/'):-1]#網址當排序依據
        
        print ctype,cbook,cname,len(cbn),'_',cbn,cdata#,clink#,soup.select('.m-boxListBookProduct__item')[a]
        #print 'check',check
        #break
        
        #data處理
        if len(cdata) < 3:#無data
            cdata = '0000/00/00'#填入data
            check = 4#
        data = cdata
        while listdata.count(data):#重複data判斷
            data = data[:-2] + str(int(data[-2:]) + 1).rjust(2,'0')#尾數+1_十位數填0
        listdata.append(data)
        
        book=cbook
        btype=ctype
        name=cname
        bn=cbn
        blink=clink
        
        #【セット】
        #【フルカラー】（単話）
        #寫入dict
        if (len(bn)>0) or (check==3) or (u'【セット】' in book):
            if len(bn)>0:
                book=book+u'['+bn+u']'
            book=book+'_!'+blink
        #print book
        
        if check==1:#單行本
            dict1.setdefault(data,book)
        elif check==2:#單話
            dict2.setdefault(data,book)
        else:#多作者
            book=u'['+name+u']['+btype+u']_'+book
            dict3.setdefault(data,book)
        
        #club=cclub.strip().strip(u'\n').replace('\n','')
        continue
    print '.'
    #print '========'
    #return

########

key=key.lower()
pn=0

tmp=soup.select('.m-boxPagenation__txt')[0].text
pn=tmp[tmp.find(u'全')+1:-1]#資料筆數
print pn

'''
アダルトコミック
美少女ノベル・官能小説
アダルト写真集・雑誌
#美少女ノベル_コミック雑誌
#小說與寫真無法區分，皆不記錄。
含小說_ぽるのいぶき
含寫真_奥森ボウイ

単行本
単話
コミック雑誌
#【セット】

#標題切斷範例_奥森ボウイ
#多作者_LINDA
#'''

#資料筆數_是否數字
if pn>0:
    print 'ok'
    #print soup.find_all('b')
    
    if pn > pnum:
        print 'BIG'
    
    fout = open('output/'+key.decode('utf8') + '_dmmv1.txt', 'w')#寫入模式開檔
    fout.write('dmm\n')#toranoana
    print key , pn , 'num\n========v1'
    time.sleep(1)
    fout.write('!' + key + '\n!總筆數' + str(pn).encode('utf8') +'_'+ strftime("%Y/%m/%d,%H:%M")+'->')
    
    p = 0#頁
    #建空輸出用字典與陣列
    dict1={}#單行本
    dict2={}#單話
    dict3={}#多作者
    listdata = []
    bl=0#bl作品數
    
    #資料處理
    while (int(pn) - p * pnum) > 0:
        p = p + 1
        print 'page:' + str(p)
        soup = next(p)#頁
        #print soup.select('.TBLdtil')[2]
        findbook(soup)#資料處理
        time.sleep(1)
        #sys.exit()################
    
    #日期排序
    listdata.sort()
    
    fout.write(strftime("%H:%M")+'\n')
    
    temp = ''
    #dict1_單行本
    fout.write('==book_' + str(len(dict2)) +'_單行本\n')
    save(dict1)
    #dict2_單話
    fout.write('==sbook_' + str(len(dict1)) +'_單話\n')
    save(dict2)
    #dict3_多作者
    fout.write('==other_' + str(len(dict1)) +'_多作者\n')
    save(dict3)
    
    #sys.exit()################
    fout.close()
    print 'ok'
elif pn:
    print 'No Date'

#結束讀秒
x=3
while x!=0:
    print x,'..',
    x=x-1
    time.sleep(1)
raw_input("\nPress Any Key To Exit")

85
ok
BIG
奥森ボウイ 85 num
========v1
page:1
2 単話 俺得修学旅行〜男は女装した俺だけ！ 奥森ボウイ 2 _ 10 b278agphg00378
3 単話 【セット】俺得修学旅行〜男は女装した俺だけ 奥森ボウイ 1 _ 2 b266agrph00295
5 単話 パイパイちょうだい！！〜六畳一間でミルクをちゅっちゅ〜 奥森ボウイ_赤髭 1 _ 2 b924akgky01101
8 単話 夏色バズー 奥森ボウイ 0 _  b336afjcb00458
11 単話 パイパイちょうだい！！〜六畳一間でミルクをちゅっちゅ 奥森ボウイ_赤髭 1 _ 2 b924akgky00894
17 単話 【セット】鬼畜美少女狩り〜禁断の 奥森ボウイ 0 _  b266agrph00180
19 単話 踊る誘惑受付嬢〜ありえない腰使い〜 奥森ボウイ 0 _  b924akgky00838
20 単行本 今回はご縁ありました、ということ 奥森ボウイ 0 _  b216afjm00389
21 単話 ちっぱいフーゾク嬢と野外えっち！ 奥森ボウイ_kupa 0 _  b604atppn00003
25 単話 トナリノ 奥森ボウイ 0 _  b336afjcb00388
28 単話 情熱は踊る 奥森ボウイ 0 _  b336afjcb00366
30 単話 鬼畜美少女狩り〜禁断の復 奥森ボウイ 1 _ 4 b278agphg00268
.
page:2
1 単行本 放課後ふたりぼ 奥森ボウイ 0 _  b257bdmmg00243
3 単話 スタンド・バイ・ミ 奥森ボウイ 0 _  b257bdmmg00228
5 単話 俺の彼女はすーぱーぷに 奥森ボウイ 0 _  b336afjcb00341
7 単話 感じて、カテキョ〜いつやるの？今でしょ！〜 奥森ボウイ_赤髭 1 _ 2 b924akgky00477
8 単話 マンマンちょうだい！！〜義母さんは、同級生！？〜 奥森ボウイ_赤髭 1 _ 2 b924akgky00407
9 単話 マンマンちょうだい！！〜義母さんは、同級生！？ 奥森ボウイ_赤髭 1 _ 2 b924akgky00548
12 単話 演じて感じ 奥森ボウイ 0 _  b336afjcb00320
15 単話 グラドルが堕ちた 奥森ボウイ 

''

In [89]:
# -*- coding: utf-8 -*- 
import urllib , requests , sys ,string ,time
from bs4 import BeautifulSoup
from bs4 import SoupStrainer
from time import gmtime, strftime

fkey = open('key.txt', 'r')
key = fkey.readline()#key=作者
fkey.close()

c='''
<li class="m-boxListBookProductTmbSub m-boxListBookTmbSubInfo--series"><span class="m-boxListBookTmbSubInfo__txt m-boxListBookTmbSubInfo--series__txt"><a href="http://book.dmm.co.jp/series/?floor=Abook&series_id=559179">1～2巻</a></span></li>
                <li class="m-boxListBookProductTmbSub m-boxListBookProductTmbSub--price">
                    <span class="m-boxListBookProductTmbSub__txt m-boxListBookProductTmbSub--price__txt">
                        324円                    </span>
                </li>
'''

#key=作者
key2 = urllib.quote(key.decode('utf8').encode('EUC_JP'))#當sjis輸出utf8的url
key3 = urllib.unquote(key2.decode('shift_jis').encode('utf8'))#utf8的url翻sjis
#網址用
key4 = urllib.quote(key)
'''
http://www.dmm.co.jp/search/=
/searchstr=%E5%A5%A5%E6%A3%AE%E3%83%9C%E3%82%A6%E3%82%A4
/analyze=V1EBD1YFUAM_#可省略?
/limit=30#頁顯示數量
/n1=FgRCTw9VBA4GAFNXXF0_
/sort=date#排序
/view=text#檢視模式_有無
/page=2#頁_P1沒有
/#尾
'''
page=1
link="http://www.dmm.co.jp/search/=/\
searchstr="+key4+"/limit=30/n1=FgRCTw9VBA4GAFNXXF0_/sort=date/"
#link='http://book.dmm.co.jp/detail/b924akgky01101/'
#標題與話數_http://book.dmm.co.jp/detail/b333afjpc00714/
if page>1:
    link=link+'page='+str(page)+'/'

res = requests.get(link)
res.encoding =  res.apparent_encoding#亂碼處理
r= res.text
#r=r[r.find(u'<td id="mu">'):r.find(u'<!-- /mu --></td>')+len(u'<!-- /mu --></td>')]#+'</td>'#搜尋結果擷取
r=r[r.find(u'</div><![endif]-->')+len(u'</div><![endif]-->'):r.find(u'<div id="footer">')]#+'</td>'#搜尋結果擷取
#r='<html><body><table>'+r+'</table></body></html>'
#r=r[r.find(u'<!-- MAIN AREA -->'):r.rfind(u'</table>')+len(u'</table>')]#搜尋結果擷取
#print r

#only_a_tags = SoupStrainer(id="mu")#縮小檢索範圍
#soup = BeautifulSoup(r,"html.parser")#,  parse_only=only_a_tags)
soup = BeautifulSoup(r,"lxml")#,  parse_only=only_a_tags)

#'''
print len(res.text),'\n',len(str(soup))#,len(str(sou))
#print soup,str(soup)
tmp=soup.select('.m-boxPagenation__txt')[0].text
pn=tmp[tmp.find(u'全')+1:-1]#搜尋筆數
print pn
if len(str(soup))>100:
    tmp=soup.select('.m-boxPagenation__txt')[0].text
    pn=tmp[tmp.find(u'全')+1:-1]#搜尋筆數
    print pn

a=0
for n in soup.select('.m-boxListBookProduct__item'):
    cbook=n.select('.m-boxListBookProductTmb__ttl')[0].text
    ctype=n.select('.m-boxListGenreIco.m-boxListGenreIco--large')[0].text
    cbn=''#卷
    cname=n.select('.m-boxListBookProductTmb__linkAuthor')[0].text.strip().strip(u'\n')
    
    #作品頁_1標題2作者3系列
    if n.select('.m-boxListBookProductTmbSub.m-boxListBookTmbSubInfo--series'):
        cbn=n.select('.m-boxListBookProductTmbSub.m-boxListBookTmbSubInfo--series')[0].text
    #cbn=n.select('.m-boxListBookProductTmbSub.m-boxListBookTmbSubInfo--series')#[0].text
    if (u'美少女ノベル' == ctype) or (u'コミック雑誌' == ctype):
        print ctype,'null'
    else:
        print a,cbook,ctype,len(ctype),cname,cbn#,soup.select('.m-boxListBookProduct__item')[a]
    a=a+1
#print key4
print '\n.'
#'''

'''
dbook=soup.select('.m-boxDetailProductInfoMainList__description__list')[1].select('a')[0].text
for d in soup.select('.m-boxDetailProductInfoMainList__description__list')[0].select('a'):
    print d.text
print dbook
#'''

106547 
61378
85
85
コミック雑誌 null
1 俺得修学旅行～男は女装し... 単話 2 奥森ボウイ 1～10巻
2 【セット】俺得修学旅行～... 単話 2 奥森ボウイ 1～2巻
コミック雑誌 null
4 パイパイちょうだい！！～... 単話 2 奥森ボウイ 他 1～2巻
コミック雑誌 null
コミック雑誌 null
7 夏色バズーカ（単話） 単話 2 奥森ボウイ 
コミック雑誌 null
コミック雑誌 null
10 パイパイちょうだい！！～... 単話 2 奥森ボウイ 他 1～2巻
美少女ノベル null
コミック雑誌 null
美少女ノベル null
美少女ノベル null
美少女ノベル null
16 【セット】鬼畜美少女狩り... 単話 2 奥森ボウイ 
コミック雑誌 null
18 踊る誘惑受付嬢～ありえな... 単話 2 奥森ボウイ 
19 今回はご縁ありました、と... 単行本 3 奥森ボウイ 
20 ちっぱいフーゾク嬢と野外... 単話 2 奥森ボウイ 他 
コミック雑誌 null
コミック雑誌 null
コミック雑誌 null
24 トナリノ女（単話） 単話 2 奥森ボウイ 
コミック雑誌 null
コミック雑誌 null
27 情熱は踊る♪（単話） 単話 2 奥森ボウイ 
コミック雑誌 null
29 鬼畜美少女狩り～禁断の復... 単話 2 奥森ボウイ 1～4巻

.


"\ndbook=soup.select('.m-boxDetailProductInfoMainList__description__list')[1].select('a')[0].text\nfor d in soup.select('.m-boxDetailProductInfoMainList__description__list')[0].select('a'):\n    print d.text\nprint dbook\n#"